1. Engineer your features. Here you do not have them for free. You need to think of possible ways for transforming the collected data into meaningful features. For some ideas, consider traditional features such as texture features, color features, bags of visual words or more powerful ones involving CNNs. If you cannot think of anything, talk to the professor for some ideas.

2. Propose classification techniques to solve the problem. Suggestions here are the CNN directly, or SVMs/Random Forests allied with CNNs through the use of transfer learning.

3. Consider using data augmentation in the training (what about in the testing as well?)

4. Observation: You are free to use any solution to help you extract the features at this point.

5. Report all of your results for the validation and test data. The labels for the test will be released one week before the deadline.

In [314]:
#imports
from glob import glob
from skimage import io
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.optimizers import SGD
from keras.utils import np_utils
import numpy as np

In [315]:
#Definicoes
dataFolder = 'data/'
trainFolder = dataFolder + 'train/'
validationFolder = dataFolder + 'val/'
testFolder = dataFolder + 'test/'

numberOfClasses = 82

In [316]:
#Classes

class LeNet:
    @staticmethod
    def build(width, height, depth, classes, weightsPath=None):
        # initialize the model
        model = Sequential()
        # first set of CONV => RELU => POOL
        model.add(Convolution2D(20, 5, 5, border_mode="same", input_shape=(height, width, depth)))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), dim_ordering="th"))
        # second set of CONV => RELU => POOL
        model.add(Convolution2D(50, 5, 5, border_mode="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), dim_ordering="th"))
        # set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(500))
        model.add(Activation("relu"))
 
        # softmax classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        # if a weights path is supplied (inicating that the model was pre-trained), then load the weights
        if weightsPath is not None:
            model.load_weights(weightsPath)
 
        # return the constructed network architecture
        return model

In [317]:
#Funcoes

#Retorna o vetor da imagem dado o nome do seu arquivo
def le_imagem(name):
    return io.imread(name,plugin='matplotlib') 

#Retorna o vetor da imagem e a classe dado o caminho do arquivo
def le_imagem_e_classe(name):
    img = le_imagem(name)
    classe = np_utils.to_categorical(int(name.split('/')[2].split('_')[0]), numberOfClasses)
    return img, classe

#Retorna o caminho de todas as imagens dado a pasta
def nome_das_imagens(pasta):
    return glob(pasta + '*')

#Retorna o numero correspondente a predicao
def categorical_to_number(vector):
    maior = 0
    for i in range(len(vector)):
        if vector[i] > vector[maior]:
            maior = i
    return maior

In [318]:
#Le uma imagem
img, classe1 = le_imagem_e_classe(trainFolder + '00_0004.jpg')
largura = img.shape[1]
altura = img.shape[0]
profundidade = img.shape[2]

In [319]:
#Cria o dataset
X = np.array([img])
y = np.array([classe1])

In [320]:
print(classe1)

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [321]:
#Treina essa unica imagem
model = LeNet.build(width=largura, height=altura, depth=profundidade, classes=numberOfClasses)
opt = SGD(lr=0.01)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
model.fit(X,y)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(20, (5, 5), input_shape=(200, 200,..., padding="same")`
  if __name__ == '__main__':
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), data_format="channels_first")`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(50, (5, 5), padding="same")`
  del sys.path[0]
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), data_format="channels_first")`
  from ipykernel import kernelapp as app


Epoch 1/1
1/1 [==============================] - 9s 9s/step - loss: 16.1181 - acc: 0.0000e+00


In [322]:
#Testa o modelo treinado
print(categorical_to_number(model.predict(X)[0]))

33
